<a href="https://colab.research.google.com/github/pogpog/sentiment-analysis/blob/main/Fine_tune_ModernBert_sentiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -q "torch==2.5.0" "torchvision==0.20.0"
%pip install -q "setuptools<71.0.0" scikit-learn

%pip install -q --upgrade \
  "datasets==3.1.0" \
  "accelerate==1.2.1" \
  "hf-transfer==0.1.8"

%pip install -q "git+https://github.com/huggingface/transformers.git@6e0515e99c39444caae39472ee1b2fd76ece32f1" --upgrade

In [ ]:
from datasets import load_dataset
from datasets.arrow_dataset import Dataset
from datasets.dataset_dict import DatasetDict, IterableDatasetDict
from datasets.iterable_dataset import IterableDataset

In [ ]:
# Dataset id from huggingface.co/dataset
# dataset_id = "argilla/synthetic-domain-text-classification"
dataset_id = "pogpog/data-sentiments-01"

# Load raw dataset
train_dataset = load_dataset(dataset_id, split='train')

README.md:   0%|          | 0.00/465 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/9.30k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/250 [00:00<?, ? examples/s]

In [ ]:
shuffled_dataset = train_dataset.shuffle(seed=42)

In [ ]:
split_dataset = shuffled_dataset.train_test_split(test_size=0.1)
split_dataset['train'][1:3]

{'text': ['When does your office close?', 'pay right now?'],
 'labels': [2, 2],
 '__index_level_0__': [120, 168]}

In [ ]:
split_dataset['train'][10]

{'text': 'good night', 'labels': 2, '__index_level_0__': 130}

In [ ]:
# print(split_dataset['train'].features)

In [ ]:
split_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', '__index_level_0__'],
        num_rows: 225
    })
    test: Dataset({
        features: ['text', 'labels', '__index_level_0__'],
        num_rows: 25
    })
})

In [ ]:
from transformers import AutoTokenizer

# Model id to load the tokenizer
model_id = "answerdotai/ModernBERT-base"

# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Tokenize helper function
def tokenize(batch):
    return tokenizer(batch['text'],
                     truncation=True,
                     padding='max_length',
                     max_length=128,
                     return_tensors="pt")

# Tokenize dataset
if "label" in split_dataset["train"].features.keys():
    split_dataset =  split_dataset.rename_column("intent", "labels") # to match Trainer
tokenized_dataset = split_dataset.map(tokenize, batched=True, remove_columns=["text"])

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.13M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

Map:   0%|          | 0/225 [00:00<?, ? examples/s]

Map:   0%|          | 0/25 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSequenceClassification

# Model id to load the tokenizer
model_id = "answerdotai/ModernBERT-base"

# Prepare model labels - useful for inference
labels = tokenized_dataset["train"].features["labels"].names
num_labels = len(labels)
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

# Download the model from huggingface.co/models
model = AutoModelForSequenceClassification.from_pretrained(
    model_id, num_labels=num_labels, label2id=label2id, id2label=id2label,
)

config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/599M [00:00<?, ?B/s]

Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at answerdotai/ModernBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
print(labels)
print("Total labels", len(labels))

['anger', 'negative', 'neutral', 'positive']
Total labels 4


In [ ]:
from transformers import Trainer, TrainingArguments

NUM_EPOCHS = 3

# Define training args
training_args = TrainingArguments(
    output_dir= "ModernBERT-domain-classifier",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=2,
    learning_rate=5e-5,
    num_train_epochs=NUM_EPOCHS,
    weight_decay=0.01,
    bf16=True, # bfloat16 training
    optim="adamw_torch_fused", # improved optimizer
    # logging & evaluation strategies
    logging_strategy="steps",
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    # report_to="wandb",
    report_to="none",
)

# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"]
)
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,1.276324
2,No log,0.114976
3,No log,0.181216


TrainOutput(global_step=171, training_loss=0.08503542448344983, metrics={'train_runtime': 94.4052, 'train_samples_per_second': 7.15, 'train_steps_per_second': 1.811, 'total_flos': 57503683814400.0, 'train_loss': 0.08503542448344983, 'epoch': 3.0})

In [ ]:
import numpy as np

predictions = trainer.predict(tokenized_dataset["test"])

# Process the prediction results (predictions, label_ids, metrics)
predicted_labels = np.argmax(predictions.predictions, axis=1)


In [ ]:
print(type(predicted_labels))
print(predicted_labels.shape)
print(predicted_labels[0])

<class 'numpy.ndarray'>
(25,)
2


In [ ]:
for pl in predicted_labels:
    print(id2label[str(pl)])

    predicted_label = id2label[str(pl)]
    # example_data = [split_dataset['train'][0]]
    example_data = id2label[str(split_dataset['test'][0]['labels'])]

    print(f"Predicted Label: {predicted_label}")
    print(f"Actual Label: {example_data}")
    print()

neutral
Predicted Label: neutral
Actual Label: neutral

neutral
Predicted Label: neutral
Actual Label: neutral

anger
Predicted Label: anger
Actual Label: neutral

negative
Predicted Label: negative
Actual Label: neutral

negative
Predicted Label: negative
Actual Label: neutral

neutral
Predicted Label: neutral
Actual Label: neutral

neutral
Predicted Label: neutral
Actual Label: neutral

neutral
Predicted Label: neutral
Actual Label: neutral

positive
Predicted Label: positive
Actual Label: neutral

negative
Predicted Label: negative
Actual Label: neutral

neutral
Predicted Label: neutral
Actual Label: neutral

positive
Predicted Label: positive
Actual Label: neutral

neutral
Predicted Label: neutral
Actual Label: neutral

neutral
Predicted Label: neutral
Actual Label: neutral

neutral
Predicted Label: neutral
Actual Label: neutral

negative
Predicted Label: negative
Actual Label: neutral

neutral
Predicted Label: neutral
Actual Label: neutral

negative
Predicted Label: negative
Actua

In [ ]:
accuracy = np.mean(predicted_labels == predictions.label_ids)
print(f"Accuracy: {accuracy}")


Accuracy: 0.92


# Notes

## Accuracy by num epochs

Data: `argilla/synthetic-domain-text-classification`

Result by by num epochs

| Epochs | Acc |
| ------ | --- |
| 1      | 88% |
| 2      | 92% |
| **3**      | **95%** * |
| 4      | 90% |

-----------------------------------------------

Data: `hopper-intents` all data

Result by num epochs

| Epochs | Acc |
| ------ | --- |
| 1      | 78% |
| 2      | 81% * |
| 3      | 79% |
| 4      | 80% |

-----------------------------------------------

Data: `hopper-intents` top-10 labels

Result by num epochs

| Epochs | Acc |
| ------ | --- |
| 1      | 87% |
| **2**      | **88%** * |
| 3      | 86% |
| 4      | 84%